# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Fri Sep 17 20:30:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   58C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 Samsung Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_samsung.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,삼성전자,5930,매일경제,2020010104,무르익는 5G·폴더블폰…4년만에 반등 예감,http://news.mk.co.kr/newsRead.php?no=327&year=...,◆ 2020 경제기상도 / 업종별 전망 (스마트폰) ◆\r\n\n\n 올해는 5세대...,2020-01-02,4,55500,56000,55000,55200,12993228,-0.010753,-1,0,경제 기상도 업종 전망 스마트폰 올해 세대 이동통신 확대 스마트폰 판매 반등 전망 ...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2020 = pos_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2020-01-02 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2020-01-02 Cosine Similarity between <최고점> & <최고점> : 0.9999999403953552
***2020-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2020-01-02 Cosine Similarity between <증대> & <감소> : 0.6517643928527832
***2020-01-02 Cosine Similarity between <호황> & <불황> : 0.7041271924972534
***2020-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2020-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2020-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2020-01-02 Cosine Similarity between <증대> & <감소> : 0.6517643928527832
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <호황

In [11]:
pos_lexicon_2020

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,중요한,가장,0.537125,1
1,2020-01-02,1,증대,확보,0.516144,1
2,2020-01-02,1,증대,확대,0.519324,1
3,2020-01-02,1,증대,증가,0.694643,3
4,2020-01-02,1,최고점,최고점,1.000000,1
...,...,...,...,...,...,...
31997,2021-01-04,7444,경신,기록,0.522991,2
31998,2021-01-04,7444,본격적인,본격,0.610438,1
31999,2021-01-04,7444,중요한,가장,0.537125,1
32000,2021-01-04,7444,증대,확대,0.519324,1


In [12]:
pos_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/pos_samsung_2020.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2020['news_word'])))

a

['단점',
 '배송물량',
 '윤여삼',
 '펙티브',
 '엠에스',
 '유지담',
 '삼성고',
 '극복',
 '에이킨검프',
 '요구',
 '흐트러트리',
 '강창진',
 '고취',
 '응우옌티킴티',
 '송곡여고',
 '노점온도',
 '헥슬란트',
 '리메드',
 '전윤종',
 '인국공',
 '딴마음',
 '엠피대산엠',
 '조보람',
 '수펙스홀',
 '에어드랍',
 '루컴즈',
 '향상',
 '황규진',
 '미탁스',
 '침구',
 '확보',
 '포켓린트',
 '다대고',
 '스냅드래건',
 '물량이긴',
 '론칭행사',
 '마련',
 '소배심',
 '메디프론디비티',
 '박도수',
 '증진',
 '무림제지',
 '솔레니얼',
 '몸치장',
 '딜라이트숍',
 '자공시',
 '메디앙스',
 '스트레티',
 '강세',
 '정찬',
 '본격',
 '언택트',
 '기소진',
 '지앤비밴딩',
 '거버넌스위',
 '시너지',
 '가장',
 '폰지사기',
 '청소컴넷',
 '량젠쟝',
 '실증특례',
 '완판되자',
 '파워텔',
 '솔티드인솔',
 '재택명령',
 '올리콘',
 '해법',
 '글루가',
 '레드브릭',
 '중궈중처',
 '서스테널리틱스',
 '서명진',
 '경제원',
 '문제해결',
 '임영우',
 '펜데믹',
 '아랍헬스',
 '어시스턴드',
 '프롭테크',
 '루브리컨츠',
 '폴드블폰',
 '텍옵트론텍',
 '빅테크',
 '박유악',
 '불황',
 '폐쇄하거',
 '프터스',
 '준감위',
 '디지털신호',
 '토트넘홋스퍼',
 '레졸루',
 '서형준',
 '나성준',
 '어오시에이츠',
 '에반블래스',
 '송태성',
 '스맨트',
 '연천군',
 '오벤터스',
 '웃돈',
 '므와바',
 '나녀온',
 '엔드플레이트',
 '구정우',
 '웃돌',
 '라디언트',
 '입관식',
 '비티봇',
 '한게임커뮤니케이션',
 '폴딩방식',
 '스위덴',
 '차재진',
 '미래한국당',
 '피앤티디',
 '윤채연',
 '팽행하',
 '거버넌스

In [14]:
b = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['추천', '약세', '밑돈', '최고점', '불황', '우세', '추진', '단점', '실패', '갈란츠', '쎄미켐동진쎄미켐']

In [15]:
c = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['약세',
 '감소',
 '오벤터스',
 '단점',
 '적극',
 '쎄미켐동진쎄미켐',
 '셰도보팅',
 '추천',
 '밑돈',
 '최고점',
 '텍옵트론텍',
 '불황',
 '추진',
 '인터플랙스',
 '웃돌',
 '실패',
 '갈란츠',
 '선봬높',
 '재택명령',
 '우세',
 '한게임커뮤니케이션',
 '브로앤팁스',
 '스위덴',
 '것이므',
 '송출되',
 '맞소송',
 '증가',
 '홈컨시어']